Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [6]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [7]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [8]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[5, 5, 1, 16]
[16, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]


In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.825204
Minibatch accuracy: 12.5%
Validation accuracy: 11.4%
Minibatch loss at step 50: 1.507004
Minibatch accuracy: 50.0%
Validation accuracy: 50.9%
Minibatch loss at step 100: 1.128740
Minibatch accuracy: 50.0%
Validation accuracy: 69.5%
Minibatch loss at step 150: 1.520594
Minibatch accuracy: 56.2%
Validation accuracy: 70.5%
Minibatch loss at step 200: 1.080940
Minibatch accuracy: 68.8%
Validation accuracy: 75.8%
Minibatch loss at step 250: 0.943564
Minibatch accuracy: 75.0%
Validation accuracy: 78.6%
Minibatch loss at step 300: 0.320749
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 350: 0.827811
Minibatch accuracy: 75.0%
Validation accuracy: 79.6%
Minibatch loss at step 400: 0.968733
Minibatch accuracy: 68.8%
Validation accuracy: 81.3%
Minibatch loss at step 450: 0.329783
Minibatch accuracy: 93.8%
Validation accuracy: 81.4%
Minibatch loss at step 500: 0.315037
Minibatch accuracy: 87.5%
Validation accuracy: 79.7%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [20]:
stride = 2
num_steps = 2000
learning_rate = 0.05
decay_rate = 0.75
decay_steps = 1000
patch_size = 2
batch_size = 32

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    layer_1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer_1_biases = tf.Variable(tf.zeros([depth]))
    
    pool_1_weights = tf.Variable(
        tf.truncated_normal([image_size // stride * image_size // stride * num_channels,
                             image_size // stride * image_size // stride * num_channels], stddev = 0.1))
    pool_1_biases = tf.Variable(
        tf.constant(1.0, shape = [image_size // stride * image_size // stride * num_channels]))
    
    layer_2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer_2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer_3_weights = tf.Variable(tf.truncated_normal(
        [image_size // (stride ** 2) * image_size // (stride ** 2) * depth, num_hidden], stddev=0.1))
    layer_3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer_4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer_4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    def model(data): 
        pool = tf.nn.max_pool(data, [1, patch_size, patch_size, 1], [1, stride, stride, 1], padding = 'SAME')
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.reshape(
            tf.matmul(reshape, pool_1_weights) + pool_1_biases,
            [shape[0], shape[1], shape[2], shape[3]])
        
        pool = tf.nn.max_pool(hidden, [1, patch_size, patch_size, 1], [1, stride, stride, 1], padding = 'SAME')
        hidden = tf.nn.relu(pool + layer_2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer_3_weights) + layer_3_biases)
        return tf.nn.relu(tf.matmul(hidden,layer_4_weights) + layer_4_biases)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(model(tf_train_dataset), tf_train_labels))
    
    global_step = tf.Variable(0, trainable=False)
    rate = tf.train.exponential_decay(learning_rate, global_step, decay_rate, decay_steps)        
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss, global_step=global_step)
    
    train_guess = tf.nn.softmax(model(tf_train_dataset))
    valid_guess = tf.nn.softmax(model(tf_valid_dataset))
    test_guess = tf.nn.softmax(model(tf_test_dataset))
    print("Graph built")

Graph built


In [21]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initizalized")
    for step in range(num_steps):
        start = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[start:(start + batch_size),:,:,:]
        batch_labels = train_labels[start:(start + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_guess],
                                       feed_dict=feed_dict)
        if step % 100 == 0:
            print("%d: Loss = %f, Validation accuracy = %.1f" %
                 (step, l, accuracy(valid_guess.eval(), valid_labels)))
    print("Test accuracy = %.1f%%" % accuracy(test_guess.eval(), test_labels))

Initizalized
0: Loss = 5.075458, Validation accuracy = 10.0
100: Loss = 2.305073, Validation accuracy = 10.0
200: Loss = 2.307214, Validation accuracy = 10.0
300: Loss = 2.304102, Validation accuracy = 10.0
400: Loss = 2.306327, Validation accuracy = 10.0
500: Loss = 2.293926, Validation accuracy = 10.0
600: Loss = 2.306099, Validation accuracy = 10.0
700: Loss = 2.304226, Validation accuracy = 10.0
800: Loss = 2.306589, Validation accuracy = 10.0
900: Loss = 2.311363, Validation accuracy = 10.0
1000: Loss = 2.304235, Validation accuracy = 10.0
1100: Loss = 2.301548, Validation accuracy = 10.0


KeyboardInterrupt: 

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [50]:
num_steps = 200000 / batch_size
batch_size = 32
patch_size = 5
depth = 16
num_hidden = 64
init_learning_rate = 0.025
decay_rate = 0.75
decay_steps = 2000

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights1 = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases1 = tf.Variable(tf.zeros([depth]))
    layer1_weights2 = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases2 = tf.Variable(tf.zeros([depth]))
    
    layer2_weights1 = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases1 = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer2_weights2 = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
      [2 * image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights1, [1, 2, 2, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases1)
        conv2 = tf.nn.conv2d(data, layer1_weights2, [1, 2, 2, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer1_biases2)
        
        conv1 = tf.nn.conv2d(hidden1, layer2_weights1, [1, 2, 2, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer2_biases1)
        conv2 = tf.nn.conv2d(hidden2, layer2_weights2, [1, 2, 2, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases2)
        
        shape1 = hidden1.get_shape().as_list()
        reshape1 = tf.reshape(hidden1, [shape1[0], shape1[1] * shape1[2] * shape1[3]])
        shape2 = hidden2.get_shape().as_list()
        reshape2 = tf.reshape(hidden2, [shape2[0], shape2[1] * shape2[2] * shape2[3]])
        combined = tf.concat(1, [reshape1, reshape2])
        hidden = tf.nn.relu(tf.matmul(combined, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    def model_with_dropout(data):
        conv1 = tf.nn.conv2d(data, layer1_weights1, [1, 2, 2, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases1)
        conv2 = tf.nn.conv2d(data, layer1_weights2, [1, 2, 2, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer1_biases2)
        
        conv1 = tf.nn.conv2d(hidden1, layer2_weights1, [1, 2, 2, 1], padding='SAME')
        hidden1 = tf.nn.dropout(tf.nn.relu(conv1 + layer2_biases1), 0.5)
        conv2 = tf.nn.conv2d(hidden2, layer2_weights2, [1, 2, 2, 1], padding='SAME')
        hidden2 = tf.nn.dropout(tf.nn.relu(conv2 + layer2_biases2), 0.5)
        
        shape1 = hidden1.get_shape().as_list()
        reshape1 = tf.reshape(hidden1, [shape1[0], shape1[1] * shape1[2] * shape1[3]])
        shape2 = hidden2.get_shape().as_list()
        reshape2 = tf.reshape(hidden2, [shape2[0], shape2[1] * shape2[2] * shape2[3]])
        combined = tf.concat(1, [reshape1, reshape2])
        hidden = tf.nn.relu(tf.matmul(combined, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model_with_dropout(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    global_step = tf.Variable(0, trainable=False)
    rate = tf.train.exponential_decay(init_learning_rate, global_step, decay_steps, decay_rate)
    optimizer = tf.train.GradientDescentOptimizer(rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [51]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initizalized")
    for step in range(num_steps):
        start = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[start:(start + batch_size),:,:,:]
        batch_labels = train_labels[start:(start + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction],
                                       feed_dict=feed_dict)
        if step % 100 == 0:
            print("%d: Loss = %f, Validation accuracy = %.1f" %
                 (step, l, accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy = %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initizalized
0: Loss = 5.146671, Validation accuracy = 10.0
100: Loss = 1.936816, Validation accuracy = 53.0
200: Loss = 1.233626, Validation accuracy = 72.5
300: Loss = 1.063901, Validation accuracy = 77.9
400: Loss = 1.109857, Validation accuracy = 79.6
500: Loss = 0.985595, Validation accuracy = 79.5
600: Loss = 1.164552, Validation accuracy = 81.0
700: Loss = 0.611234, Validation accuracy = 81.5
800: Loss = 1.294408, Validation accuracy = 81.6
900: Loss = 0.504283, Validation accuracy = 82.2
1000: Loss = 0.911111, Validation accuracy = 82.9
1100: Loss = 0.533211, Validation accuracy = 83.1
1200: Loss = 0.738193, Validation accuracy = 83.3
1300: Loss = 0.882386, Validation accuracy = 83.1
1400: Loss = 0.626229, Validation accuracy = 83.9
1500: Loss = 0.494324, Validation accuracy = 83.7
1600: Loss = 0.992579, Validation accuracy = 83.9
1700: Loss = 1.184007, Validation accuracy = 83.8
1800: Loss = 0.685211, Validation accuracy = 84.0
1900: Loss = 0.680089, Validation accuracy = 84.1